In [155]:
import os
import re
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageDraw  
from PIL import ImageFont
from moviepy.editor import *
import moviepy.audio.fx.all as afx

class psychoCards:
    def __init__(self,xlsFn,title):
        self.dir='I:\大智小超\心理'
        self.xlsFn=xlsFn
        self.title=title
    
    def makeList(self):
        fn=os.path.join(self.dir,self.xlsFn)
        ctns=pd.read_excel(fn)
#         print(ctns['标题二'],ctns['内容'],ctns['详细'])
        lst=np.array(ctns.loc[ctns['标题二']==self.title]['标题二']).tolist()
        lst.extend(ctns.loc[ctns['标题二']==self.title]['内容'])
#         lst.extend(ctns.loc[ctns['标题二']==self.title]['详细'])
#         print(lst)
        return lst
    
    def makePics(self):
        txts=self.makeList()
        title=txts[0]
        txt='\n'.join(txts[1:])
        img=Image.open(os.path.join(self.dir,self.bg))
        
        font_title = ImageFont.truetype('C:\Windows\Fonts\msyh.ttc',30)
        font_txt = ImageFont.truetype('C:\Windows\Fonts\msyh.ttc',20)
        draw = ImageDraw.Draw(img)            
        draw.text((20,int(img.size[1]*0.2)), title, fill = '#6AB34A',font=font_title)
        draw.text((20,int(img.size[1]*0.3)), txt, fill = '#6AB34A',font=font_txt)
        img.show()
        
    def makeMovie(self):
        lst=self.makeList()
        drtns=[]        
        s=0
        f=0
        total_c=len(lst[1])
        total_time=33
        spd=total_time/total_c
        
        txt=[lst[0]]
        txt.extend(lst[1].split('\n'))
        print(txt)
        
        drtns=[]
        for t in txt:
            drtns.append(len(t)*spd)        
        drtns[0]=1
        print(drtns)
        print('正在生成影片……')
        pics=self.re_name()
        clips=[]
        k=0
        for pic in pics:
            if pic.lower()[-3:]=='jpg':
                print(pic,txt[k],f,drtns[k])
                _img=ImageClip(os.path.join(self.dir,self.title,pic)).set_fps(25).set_start(f).set_duration(drtns[k]).resize((480,640))
                f=f+drtns[k]
                k+=1         
                
                clips.append(_img)
        finalclip = CompositeVideoClip(clips)
        
        
        
        # 添加语音
        BGM=AudioFileClip(os.path.join(self.dir,self.title,self.title+'.mp3'))
        fa=BGM.set_start(5)
#         final_audio = CompositeAudioClip([BGM,finalclip.audio]).set_duration(finalclip.duration)
        mix=finalclip.set_audio(fa)
        
        out=os.path.join(self.dir,self.title,self.title+'.mp4')
        mix.write_videofile(out)
        
        print('完成影片')
    
    def re_name(self):
        print('重命名文件以排序……')
        old=os.path.join(self.dir,self.title)
        fn_list=os.listdir(old)
        ptn=r'幻灯片.*jpg'
        n=len(fn_list)
        m=0
        for fn in fn_list:
            q=re.match(ptn,fn.lower())
            if q and len(fn)<9:
                new=os.path.join(self.dir,self.title,fn[0:3]+'0'+fn[3:])
                os.rename(os.path.join(old,fn),new)
                m+=1
                
            if m>0:
                p='该文件夹内共有{0}个文件，重命名{1}个。'.format(n,m)
            else:
                p='该文件夹内共有{0}个文件，没有符合重命名条件的文件。'.format(n)
        print(p)
        new_list=os.listdir(old)
        return new_list
        
if __name__=='__main__':
    mycard=psychoCards('正面管教.xlsx','我的孩子不惩罚真的管不住怎么办')
    mycard.makeMovie()
    

['我的孩子不惩罚真的管不住怎么办', '管教是理解、鼓励和沟通。', '和孩子的沟通应当学会理解孩子的世界', '专注于解决问题，而不是惩罚。', '在过去的几十年中', '我们的社会和文化已经发生了深刻的变化', '所以', '我们教育孩子的方式也必须改变。', '惩罚短期来看也许管用', '但从长远来看', '它会造成反叛、抵制', '或不再信任自己价值的孩子。']
[1, 2.955223880597015, 4.186567164179104, 3.4477611940298507, 1.9701492537313432, 4.432835820895522, 0.4925373134328358, 3.6940298507462686, 2.462686567164179, 1.4776119402985075, 2.216417910447761, 3.201492537313433]
正在生成影片……
重命名文件以排序……
该文件夹内共有14个文件，没有符合重命名条件的文件。
幻灯片01.JPG 我的孩子不惩罚真的管不住怎么办 0 1
幻灯片02.JPG 管教是理解、鼓励和沟通。 1 2.955223880597015
幻灯片03.JPG 和孩子的沟通应当学会理解孩子的世界 3.955223880597015 4.186567164179104
幻灯片04.JPG 专注于解决问题，而不是惩罚。 8.14179104477612 3.4477611940298507
幻灯片05.JPG 在过去的几十年中 11.589552238805972 1.9701492537313432
幻灯片06.JPG 我们的社会和文化已经发生了深刻的变化 13.559701492537314 4.432835820895522
幻灯片07.JPG 所以 17.992537313432837 0.4925373134328358
幻灯片08.JPG 我们教育孩子的方式也必须改变。 18.485074626865675 3.6940298507462686
幻灯片09.JPG 惩罚短期来看也许管用 22.179104477611943 2.462686567164179
幻灯片10.JPG 但从长远来看 24.641791044776124 1.477611940298507

chunk:   0%|                                                                         | 0/627 [00:00<?, ?it/s, now=None]

Moviepy - Building video I:\大智小超\心理\我的孩子不惩罚真的管不住怎么办\我的孩子不惩罚真的管不住怎么办.mp4.
MoviePy - Writing audio in 我的孩子不惩罚真的管不住怎么办TEMP_MPY_wvf_snd.mp3


t:   0%|                                                                             | 0/789 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video I:\大智小超\心理\我的孩子不惩罚真的管不住怎么办\我的孩子不惩罚真的管不住怎么办.mp4



Moviepy - Done !
Moviepy - video ready I:\大智小超\心理\我的孩子不惩罚真的管不住怎么办\我的孩子不惩罚真的管不住怎么办.mp4
完成影片
